In [1]:
import matplotlib
from matplotlib.pyplot import *
import math
import random
import ipywidgets as W
from IPython.display import display
import time


In [2]:
def inverse_dict(D):
    if not D:
        return D, False
    if len(set(D.values())) == len(D):
        return dict( (I, k) for (k, I) in D.items() ), True
    E = {}
    for I in D.values():
        E[I] = []
    for k in D:
        E[D[k]].append(k)
    return E

In [3]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        
    def __str__(self):
        return "({0},{1})".format(self.x,self.y)    
    
    def line_to(self, q):
        return Line(self.y - q.y, -self.x + q.x, self.x* q.y - self.y * q.x)   
       
    def plus(self, q):
        return Point(self.x + q.x, self.y + q.y)
    
    def minus(self, q):
        return Point(self.x - q.x, self.y - q.y)

    def scalarmu(self, mu):
        x = mu * self.x
        y = mu * self.y
        return Point(x,y)
    
    def scalarprod(self, q):
        return self.x * q.x + self.y * q.y
    
    def reflect(self, p, q):
        t = self.minus(q).scalarprod(p.minus(q)) / p.minus(q).scalarprod(p.minus(q))
        M = q.scalarmu(1-t).plus(p.scalarmu(t))
        return M.scalarmu(2).minus(self)

    def half_reflect(self, p, q):
        t = self.minus(q).scalarprod(p.minus(q)) / p.minus(q).scalarprod(p.minus(q))
        M = q.scalarmu(1-t).plus(p.scalarmu(t))
        return M
    
    
    def is_inside(self, P):
        # the problem of intersection requires attention. 
        vals = [P[n].line_to(P[(n + 1) % len(P)]).at(self) for n in range(len(P))]
        return max(vals) <= 0 or min(vals) >= 0
    
    def perpbis(self, q): # get equation of perpendicular bisector of self and q
        dx = self.x - q.x
        dy = self.y - q.y
        a = (dx * dx + dy * dy) / 2 + q.x * dx + q.y * dy
        return Line(dx, dy, -a)
    
    def Pshrink_to(self, Plist, scale = 1):
        # scaling transform on Plist using self as origin
        def Pshrinkto(P, Q, sc):
            Px = Q.x - (Q.x - P.x ) * sc
            Py = Q.y - (Q.y - P.y ) * sc
            return Point(Px, Py)
        outlist = []
        Q = self
        for P in Plist:
            outlist.append(Pshrink(P, Q, scale))
        return outlist

Pinf = Point(10000,10000)

In [4]:
class Line():
    def __init__(self, a, b, c):
        # normalize
        root = math.sqrt(a * a + b * b)
        self.a = a / root
        self.b = b / root
        self.c = c / root
        
    def same_side(self, p, q):
        return self.at(p) * self.at(q) > 0
    
    def polygon_clip(self, poly): # returns a list of intersection points of self with poly
        n = len(poly)
        clip = []
        for q in range(n):
            p = (q + 1) % n
            if not self.same_side(poly[p], poly[q]):
                r = self.intercept(poly[p], poly[q])
                if not r == Pinf:
                    clip.append(r)
        return clip
   
    def polygon_update(self, poly): # should only be called if we are sure that
                                    # there are two intersection points (call polygon_clip first)
        n = len(poly)
        poly2 = []
        poly2.append([])
        poly2.append([])
        current = 0
        for p in range(n):
            poly2[current].append(poly[p])
            q = (p + 1) % n
            if not self.same_side(poly[p], poly[q]):
                r = self.intercept(poly[p], poly[q])
                poly2[current].append(r)
                current = (current + 1) % 2
                poly2[current].append(r)
       
        return poly2 # NB poly2 returns a LIST containing two polygons
    
    # evaluate linear function at p
    def at(self, p):
        return self.a * p.x + self.b * p.y + self.c
    
    def sum_of_distance(self, poly): # NB these signed distances are only used in determining whether                                  
                                     # a point is inside or outside a polygon, so all 'distances' will
                                     # have the same sign, which is all we need.
                                     # using sum was a crude device to avoid having to select a test vertex
        sum = 0
        n = len(poly)
        for p in poly:
            sum += self.at(p)
        return sum
  
    def intercept(self, p, q):
        """ returns the point of intersection of self with the line joining p and q"""
        s = (self.a * (p.x - q.x) + self.b * (p.y - q.y)) # determinant
        if  math.isclose(s, 0):
            print('Pinf in intercept()')
            return Pinf # point at infinity. not really implemented as yet!
        else:
            t = (self.a * p.x + self.b * p.y + self.c)/s
            return p.scalarmu(1-t).plus(q.scalarmu(t))
    
    INF = 10000
    def intercept_t(self, p, q):
        """ returns the point of intersection of self with the line joining p and q"""
        s = (self.a * (p.x - q.x) + self.b * (p.y - q.y)) # determinant
        if  math.isclose(s, 0):
            print('infinity in intercept()')
            return INF # infinity. not really implemented as yet!
        else:
            t = (self.a * p.x + self.b * p.y + self.c)/s
            return t

    def meets(self, L):
        a1, b1, c1 = self.a, self.b, self.c
        a2, b2, c2 = L.a, L.b, L.c
        det = a1 * b2 - a2 * b1
        if math.isclose(det,0):
            print('Pinf in meets()')
            return Pinf
        xfac = c1 * b2 - c2 * b1
        yfac = a1 * c2 - a2 * c1
        return Point( -xfac / det, -yfac / det)
        

In [5]:
def ranpoints(N = 24):
    plist = []
    for k in range (N):
        plist.append(Point(2 * (random.random() - 0.5), 2 * (random.random() - 0.5)))
    return plist



def barycenter(pointlist):
    n = len(pointlist)
    if n == 0:
        print('error in barycenter() - input list empty')
        return Pinf
    xlist = []
    ylist = []
    for P in pointlist:
        xlist.append(P.x)
        ylist.append(P.y)
    x = np.sum(xlist) / n
    y = np.sum(ylist) / n
    return(Point(x, y))

def polar_scatter(n_points, concentration = 2):
    """ creates a random list of points focussed on the origin"""
    plist = []
   
    for n in range(n_points):
        
        theta = 2 * math.pi * random.random()
        r     = math.exp(- concentration * random.random())
        plist.append(Point(r * math.cos(theta), r * math.sin(theta)))
    
    return plist


In [6]:
def figurehead(f):
    #standard bit of graphics  plot initialization.  call this function to save typing
    figure(figsize=(f,f))
    axes().set_aspect('equal','datalim')


def polyplot(poly,*args,**kwargs):
    # now superseded by the poly kwarg in peedraw
    poly.append(poly[0])
    peedraw(plot, poly, *args, **kwargs)
    poly.pop()


def peedraw(func, pointlist, *args, poly = False, **kwargs): 
    """plots/fills/scatters a list of Points 
     (whereas the pyplot plot function takes a list of x and a list of y co-ords)
    func is the function name
    pointlist is the  list of Points 
    poly (default False) should be set to true when the closing arc is needed (plot only)
    NB differences in var lists, particularly scatter *args
   """
    x = []
    y = []
    for p in pointlist:
        x.append(p.x)
        y.append(p.y)
    if not poly:        
        func(x,y,*args,**kwargs)
    else:
        x.append(x[0])
        y.append(y[0])
        func(x, y, *args, **kwargs)

def circle(a, b, r, npoints = 500):
    """returns a listof points forming a regular polygon with npoints sides, center (a,b), radius r"""
    C = []
    
    for n in range(npoints):
        theta =  math.pi / 2 + 2 * math.pi * n / npoints
        C.append(Point(a + r * math.cos(theta), b + r * math.sin(theta)))
                     
    return C




In [7]:
  
def area(P):
    """returns the (unsigned) area of the polygon P (list of Points)"""
    s = 0
    for j in range(len(P)):
        k = (j +1) % len(P)
        s += (P[k].y + P[j].y) * (P[k].x - P[j].x)
    return abs ( s  / 2)

def perimeter(P):
    """returns the (unsigned) perimeter of the polygon P (list of Points)"""
    sum = 0
    for j in range(len(P)):
        k = (j +1) % len(P)
        dx = P[k].x - P[j].x
        dy = P[k].y - P[j].y
        sum += math.sqrt(dx * dx + dy * dy)
    return sum

In [8]:
def next_point(P, q, Q):
    """the engine of the 'network1 algorithm"""
    L = q.perpbis(Q) # get equation of perpendicular bisector of q and Q
    clip = L.polygon_clip(P) # list of intersection points of line L and polygon P
    if len(clip) == 2: # rule out any exceptional cases. this requires looking at later +++++++ NB +++++++++++
        poly2 = L.polygon_update(P)
        #polygon_update returns two lists, with the newly computed intersection points added to each.
        
        # to determine which of the twob polgygons q is in we use signed distances from L
        a0 = L.sum_of_distance(poly2[0])
        a1 = L.sum_of_distance(poly2[1])
        a = L.at(q) # value of ax + by +c for  q = (x, y)
        
        if a * a0 > 0 and poly2[0]: # a and a0 have same sign
            return poly2[0]
        else:
            return poly2[1]
    else:
        return P


def network1(poly, seedlist):
    """ returns a dictionary indexed by the seed points of polygons (as lists of points) 
    subdividing poly ( for poly = external boundary, only rectangles have been considered so far) 
    each output polygon defines the base region round one of the points of seedlist"""
    
    Q = list(seedlist)
    cell_dict = {} # polygon coordinates
    for q in Q:
        # reset the initial polygon
        P = list(poly)
        for nextq in Q:
            if not nextq == q:
                P = next_point(P, q, nextq)
        cell_dict[q] = P
    return cell_dict

#-------------------------------------------------------------------------------------
#

In [9]:
# a handful of global settings 
sqmin = -1
sqmax =  1
SQ = [Point(x, y) for (x, y) in 
      zip([sqmax, sqmax, sqmin, sqmin], [sqmax, sqmin, sqmin, sqmax])]
# address offset constants to clarify access to prenode_list fields
pnID = 0 # the prenode ID is in its first field
ZONELOC = 1 #  the second field is the (p, j) pair: p = cell, j = position on cell boundary
XY   = 2 # the 3rd field is a Point
X    = 0 # the first item of the tuple XY
Y    = 1 # the second  "     "     "
#-----------------------------------------------------------------------------------


init_Nseeds = 24
seeds       = {}
cell_dict   = {}
zone_id     = {}
seedlist    = []
barycenters = []

In [10]:
screen = W.Output(layout=W.Layout( width = '400px', height = '400px'))
screenVB = W.VBox([screen])


Label1 = W.Label(value = 'uninitialized ')
Label2 = W.Label(value = '')
Label3 = W.Label(value = '3')
Label4 = W.Label(value = '4')
sample_parameters1 = W.HBox([Label1, Label2])
sample_parameters2 = W.HBox([Label3, Label4])
sample_parameters  = W.VBox([sample_parameters1, sample_parameters2])

sample_type = W.RadioButtons(options = ['polar', 'uniform'], 
                             value = 'polar', description = 'sample type')

show_type = W.RadioButtons(options = ['re-sample', 'iterate', 'redraw'])

concentration = W.BoundedFloatText(min = 0.1, max = 3.0, step = 0.1, 
                                   value = 1.5, description = "k value",
                                  disabled = False)


Nseeds = W.IntSlider(min = 3, max = 84, step = 1, value = init_Nseeds, description = 'N seeds', 
                     continuous_update = False)

def on_show_diagram_button_clicked(b):
    global seedlist, barycenters, seeds, cell_dict, zone_id
    if show_type.value == 're-sample' or not cell_dict:
        if sample_type.value == 'polar':
            seedlist = polar_scatter(Nseeds.value, concentration.value)
        else: seedlist = ranpoints(Nseeds.value)
    elif show_type.value == 'iterate':
        show_diagram_button.disabled = True
        seedlist = list(barycenters)
        
    cell_dict = network1(SQ, seedlist)
    seeds = dict( (k, seedlist[k]) for k in range(len(seedlist)))
    zone_id   = inverse_dict(seeds)
    barycenters = []
    for j in range(len(cell_dict)):
        barycenters.append(barycenter(cell_dict[seeds[j]]))
    show_Nseeds()

    screen.clear_output()
    with screen:
        figurehead(7)
        axis('off')
        peedraw(plot, SQ, poly = True)
        
        if show_seeds_Checkbox.value:
            peedraw(scatter, seedlist, 12, 'b' )
        
        if show_barycenters_Checkbox.value:
            for j in range(len(barycenters)):
                peedraw(scatter, barycenters, 12, 'k')
                
        for p in cell_dict:
            peedraw(plot, cell_dict[p], poly = True )
        show()
    show_diagram_button.disabled = False
        

    


show_seeds_Checkbox = W.Checkbox(value = True, description = 'show seeds')
show_barycenters_Checkbox = W.Checkbox(value = False, description = 'show centroids')
#resample_Checkbox = W.Checkbox(value = True, description = 'resample')

def on_show_seeds_Checkbox_change(change):
    global seeds_col
    if change['new'] == False:
        seeds_col = 'w'
    else:
        seeds_col = 'b'

#def on_iterate_button_clicked(b):
#    time.sleep(2)
#    show_diagram_button.disabled = False
    

show_seeds_Checkbox.observe(on_show_seeds_Checkbox_change, names = 'value')     

show_diagram_button = W.Button(description = 'show')
show_diagram_button.on_click(on_show_diagram_button_clicked)
#iterate_button = W.Button(description = 'iterate')    
#iterate_button.on_click(on_iterate_button_clicked)
#run_buttons = W.HBox([show_diagram_button, iterate_button])    

def on_sample_type_change(change):
    concentration.disabled = not concentration.disabled
    if change['new'] == 'polar':
        concentration.description = 'k value'
    else:
        concentration.description = '       '
        
sample_type.observe(on_sample_type_change, names = 'value')

def show_Nseeds(): 
    Label1.value = "CURRENT SAMPLE " + 'N seeds = '+ str(Nseeds.value) + ' (' + sample_type.value + ')'
    if sample_type.value == 'polar':
        Label1.value = Label1.value + '  ' + "  k = {:3.1f}".format(concentration.value)


bumpf1 = W.VBox([sample_parameters, 
                 sample_type, 
                 concentration, 
                 Nseeds, 
                 show_seeds_Checkbox,
                 show_barycenters_Checkbox,
                 show_type,
                 show_diagram_button])

GUI =  W.HBox([bumpf1, screenVB])

display(GUI)

In [11]:
def on_A_clicked(b):
    print(b)
A = W.Button(description = 'test')
A.on_click(on_A_clicked)
display(A)

Button(description='test', style=ButtonStyle())